# Pytorch常用代价函数示例

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.__version__

'1.6.0'

## 1. 交叉熵
交叉熵的公式为： $H(p,q)=-\sum_{i}\ P(i)logQ(i)$  其中P为真实值，Q为预测值。

当label为确定的类时，P是一个one-hot向量，交叉熵就相当于 $H(p,q)=-logQ(m)$ 其中m是正确类别下标

#### nn.CrossEntropyLoss() / F.cross_entropy()
计算过程：对outputs每一个样本进行softmax，再计算交叉熵，最后求平均

参数：

`torch.nn.CrossEntropyLoss(weight: Optional = None, ignore_index: int = -100, reduction: str = 'mean')`

`weight(Tensor)`- 为每个类别的loss设置权值，常用于类别不均衡问题。weight必须是float类型的tensor，其长度要与类别C一致，即每一个类别都要设置有weight

`ignore_index(int)`- 忽略某一类别，不计算其loss，其loss会为0,并且，在reduction='mean'或'sum'时，不会计算那一类的loss，除的时候的分母也不会统计那一类的样本。

`reduction(str)`- 默认为'mean'， 设置为'none'时，返回所有样本的loss；设置为'mean'时，返回所有样本loss的平均值；设置为'sum'时，返回所有样本loss的和

In [3]:
'''
示例1：nn.CrossEntropyLoss() / F.cross_entropy()基本用法
'''
outputs = torch.randn(3, 5) # (batch_size, class_num)
labels = torch.tensor([0, 2, 4]) # (batch_size, )
# nn.CrossEntropyLoss()
loss_fun = nn.CrossEntropyLoss() 
print('use nn.CrossEntropyLoss():', loss_fun(outputs, labels))
# F.cross_entropy()
print('use F.cross_entropy():',F.cross_entropy(outputs, labels))
# 等同于下面的计算过程
loss = 0.
probs = F.log_softmax(outputs, dim=1)  # softmax后取log
for i in range(len(labels)):
    label = labels[i].item()
    loss += -probs[i][label] # 正确类别的-log(p)相加
loss /= len(labels) # 取平均
print('no api:', loss)

use nn.CrossEntropyLoss(): tensor(1.0424)
use F.cross_entropy(): tensor(1.0424)
no api: tensor(1.0424)


In [4]:
'''
示例2：nn.CrossEntropyLoss() / F.cross_entropy()参数设置
'''
print("reduction='none':", F.cross_entropy(outputs, labels, reduction='none', ignore_index=4))
print("reduction='mean':", F.cross_entropy(outputs, labels, reduction='mean', ignore_index=4))
print("reduction='sum':", F.cross_entropy(outputs, labels, reduction='sum', ignore_index=4))

reduction='none': tensor([0.8890, 0.7958, 0.0000])
reduction='mean': tensor(0.8424)
reduction='sum': tensor(1.6848)


#### nn.NLLLoss() / F.nll_loss()
计算过程：对probs中正确类别的prob相加取平均, 所以F.cross_entropy() = F.log_softmax() + F.nll_loss()

参数：

`torch.nn.NLLLoss(weight: Optional = None, ignore_index: int = -100, reduction: str = 'mean')`

参数定义同CrossEntropyLoss

In [5]:
'''
示例3：nn.NLLLoss() / F.nll_loss()基本用法
'''
probs = F.log_softmax(outputs, dim=1)  # probs是outputs取log_softmax的结果
# nn.NLLLoss()
loss = nn.NLLLoss()
print(loss(probs,labels))
# F.nll_loss()
print(F.nll_loss(probs, labels))

tensor(1.0424)
tensor(1.0424)


## 2. 均方误差
均方误差的公式为： $loss(y,y')=\frac{1}{n}\sum(y-y')^2$ 

#### nn.MSELoss() / F.mse_loss()

参数：

`torch.nn.MSELoss(reduction: str = 'mean')`

`reduction(str)`- 默认为'mean'， 设置为'none'时，返回所有样本的loss；设置为'mean'时，返回所有样本loss的平均值；设置为'sum'时，返回所有样本loss的和

In [6]:
'''
示例4：nn.MSELoss() / F.mse_loss()基本用法
'''
# 实际值:采样自标准正态分布N~(0,1)的5个值
y = torch.randn(5)
# 预测值
pred = torch.randn(5)
print(y)
print(pred)

# nn.MSELoss()
loss = nn.MSELoss()
print(loss(pred, y))
# F.mse_loss()
print(F.mse_loss(pred, y))
# 模拟计算过程
mse = (pred - y).pow(2).mean() 
print(mse)

tensor([-0.6892,  0.2099,  1.0249, -1.4212,  0.0505])
tensor([ 1.6486, -1.1726, -0.3007, -0.7615,  1.7021])
tensor(2.4594)
tensor(2.4594)
tensor(2.4594)


## 3. KL散度（相对熵）
KL散度的公式为： $D_{KL} (p||q)=H(p,q)-H(p)=\sum_x p(x)log\frac{p(x)}{q(x)}$ 

其中p表示真实分布，q表示p的拟合分布，D(P||Q)表示当用概率分布q来拟合真实分布p时，产生的信息损耗。

#### nn.KLDivLoss() /F.kl_div()

参数：

`torch.nn.KLDivLoss(reduction: str = 'mean', log_target: bool = False)`

`reduction(str)`- 可选 none||sum||mean||batchmean，默认为'mean'，设置为'none'时，不做reduction操作；设置为'mean'时，返回output的均值；设置为'sum'时，返回output的和; 设置为batchmean时，返回总和除以batch_size的值

***一般来说，输入为单个概率分布时，reduction设置为sum，输入为多个样本的概率分布时，reduction设置为batchmean***

`log_target(bool)`[1.6.0引入]- 默认为False，指明是否target在传入时是否已经取log

注意！！！：input在传入时需要取log，如果log_target没有被设置为True，target在传入时不需要取log

In [10]:
'''
示例5：nn.KLDivLoss() /F.kl_div()基本用法
'''
target = torch.FloatTensor([0.1, 0.2, 0.7])
pred = torch.FloatTensor([0.5, 0.2, 0.3])
# nn.KLDivLoss
loss_fun = nn.KLDivLoss(reduction='sum') 
print(loss_fun(pred.log(), target))
# F.kl_div()
print(F.kl_div(pred.log(), target, reduction='sum'))
print(F.kl_div(pred.log(), target.log(), reduction='sum', log_target=True)) # 等价

#上面的计算过程等价于下面
loss = (target * ((target/pred).log())).sum()
print(loss) 

tensor(0.4322)
tensor(0.4322)
tensor(0.4322)
tensor(0.4322)


In [15]:
'''
示例6：nn.KLDivLoss() /F.kl_div()参数reduction设置
'''
target = torch.FloatTensor(([0.1, 0.2, 0.7], [0.3, 0.4, 0.3]))
pred = torch.FloatTensor(([0.5, 0.2, 0.3], [0.3, 0.3, 0.4]))
# F.kl_div()
print(F.kl_div(pred.log(), target, reduction='none'))
print(F.kl_div(pred.log(), target, reduction='sum'))
print(F.kl_div(pred.log(), target, reduction='mean'))
print(F.kl_div(pred.log(), target, reduction='batchmean'))

#batchmean相当于对每个概率分布的kl散度求平均（符合数学上的定义）
loss = (target * ((target/pred).log())).sum(dim=1).mean()
print(loss) 

tensor([[-0.1609,  0.0000,  0.5931],
        [ 0.0000,  0.1151, -0.0863]])
tensor(0.4609)
tensor(0.0768)
tensor(0.2305)
tensor(0.2305)
